# Projekt 3 - ML
### Krzysztof Kadowski
#### 2021

<b>TEMAT:</b> Estymacja zarobków developera/data scientista
<li>Predykcja zarobków pracownika IT na bazie corocznych ankiet stackoverflow. z lat 2017-2020.  Aplikacja przewidująca “ile powinienem zarabiać”.</li>
<li>Zbiór w dwóch wersjach - przeprocesowanej, gotowejdo użytku i w wersjisurowej. Gorąco zachęcam do eksplorowania danych surowych i korzystaniaze swoich pomysłów na preprocessing. Dostępne także ankiety z wcześniejszych lat pod adresem https://insights.stackoverflow.com/surveycodaje dodatkowe możliwości :)</li>

Główne cele projektu:
    <li>Machine learning</li>
    a. Obróbka i analiza danych - odrzucenie zbędnychzmiennych, przygotowaniedanych do modelowania etc.<br>
    b. Porównanie w zależności od typu problemu (regresja/klasyfikacja)różnegorodzaju modeli wraz z optymalizacją hiperparametrów<br>
    c. Zestawienie wyników z badań w formie adekwatnychwykresów i tabel.<br>
    d. Wszelkie inne dodatkowe rzeczy są na plus - np.zbadanie wpływu PCA na wyniki,weryfikacja interesujących hipotez o danych,w przypadku niektórych problemów zastosowanie zarówno podejścia regresyjnego,jak i  klasyfikacyjnego i co tamjeszcze sobie wymyślicie, możliwości zazwyczaj jest dużo :)<br>
    <li>Aplikacja</li>
    a. Prezentacja wartości biznesowej wynikającej z przygotowanegomodelu (np. poparte adekwatnymi wizualizacjami, tabelkami, możliwieinteraktywnymi). Jeśli to możliwe aplikacja powinnateż pozwalaćużytkownikowi na podanie wartości parametrów, pozwalającychuzyskać dla nich wynik najlepszego/wybranego modelu.<br>
    b. Jeśli umożliwiamy użytkownikowi podanie własnychdanych aplikacjapowinna je walidować, tj. możliwe powinno być podanietylko wartościz zakresu sensownego dla danej zmiennej.<br>
    c. Architektura aplikacji powinna wykorzystywać programowanie obiektowe.<br>

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from scipy import stats as st

from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble  # pakiet z lasami decyzyjnymi
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import ensemble
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from mlxtend import plotting
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression


plt.rcParams["figure.figsize"] = (20,6)
%matplotlib inline

In [2]:
df = pd.read_csv("survey_final.csv")
df.head()

/home/kk/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Year,Hobbyist,ConvertedComp,Country,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,JobSat,LanguageDesireNextYear,LanguageWorkedWith,OrgSize,UndergradMajor,YearsCodePro
0,2020,Yes,NaN,Germany,Microsoft SQL Server,Elasticsearch;Microsoft SQL Server;Oracle,"Developer, desktop or enterprise applications;...","Master's degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",Slightly satisfied,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,2 to 9 employees,"Computer science, computer engineering, or sof...",27
1,2020,No,NaN,United Kingdom,NaN,NaN,"Developer, full-stack;Developer, mobile","Bachelor's degree (B.A., B.S., B.Eng., etc.)",Employed full-time,Very dissatisfied,Python;Swift,JavaScript;Swift,"1,000 to 4,999 employees","Computer science, computer engineering, or sof...",4
2,2020,Yes,NaN,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN
3,2020,Yes,NaN,Albania,NaN,NaN,NaN,"Master's degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,Slightly dissatisfied,NaN,NaN,20 to 99 employees,"Computer science, computer engineering, or sof...",4
4,2020,Yes,NaN,United States,MySQL;PostgreSQL,MySQL;PostgreSQL;Redis;SQLite,NaN,"Bachelor's degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,Java;Ruby;Scala,HTML/CSS;Ruby;SQL,NaN,"Computer science, computer engineering, or sof...",8


In [3]:
df.columns 

Index(['Year', 'Hobbyist', 'ConvertedComp', 'Country',
       'DatabaseDesireNextYear', 'DatabaseWorkedWith', 'DevType', 'EdLevel',
       'Employment', 'JobSat', 'LanguageDesireNextYear', 'LanguageWorkedWith',
       'OrgSize', 'UndergradMajor', 'YearsCodePro'],
      dtype='object')

<li>Year - rok badania</li>
<li>Hobbyist - czy hobbistycznie (Yes/No -> 0/1)</li>
<li>ConvertedComp - sallary (nany odrzucamy?)</li>
<li>Country - kraj (grupowanie?)</li>
<li>DatabaseDesireNextYear - zapotrzebowanie na bazę w następnym roku - (split)</li>
<li>DatabaseWorkedWith - bazy z którymi pracował - (split)</li>
<li>DevType - typ dewelopera - (split)</li>
<li>EdLevel - edukacja (sprawdzić czy nie ma dziwnych)</li>
<li>Employment - zatrudnienie</li>
<li>JobSat - satysfakcja z pracy (sprawdzić czy nie ma dziwnych)</li>
<li>LanguageDesireNextYear - zapotrzebowanie na język - (split)</li>
<li>LanguageWorkedWith - używane języki - (split)</li>
<li>OrgSize - rozmiar organizacji (sprawdzić czy sa dziwne)</li>
<li>UndergradMajor - zakres działalności - (split)</li>
<li>YearsCodePro - ile przepracowanych lat (sprawdzić % nanów i skew - wypełnić przez mean/median)</li>

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303591 entries, 0 to 303590
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Year                    303591 non-null  int64  
 1   Hobbyist                303546 non-null  object 
 2   ConvertedComp           151172 non-null  float64
 3   Country                 302658 non-null  object 
 4   DatabaseDesireNextYear  200167 non-null  object 
 5   DatabaseWorkedWith      216942 non-null  object 
 6   DevType                 258928 non-null  object 
 7   EdLevel                 289916 non-null  object 
 8   Employment              297748 non-null  object 
 9   JobSat                  225834 non-null  object 
 10  LanguageDesireNextYear  248070 non-null  object 
 11  LanguageWorkedWith      257052 non-null  object 
 12  OrgSize                 226578 non-null  object 
 13  UndergradMajor          248486 non-null  object 
 14  YearsCodePro        

In [5]:
df.isna().sum()

Year                           0
Hobbyist                      45
ConvertedComp             152419
Country                      933
DatabaseDesireNextYear    103424
DatabaseWorkedWith         86649
DevType                    44663
EdLevel                    13675
Employment                  5843
JobSat                     77757
LanguageDesireNextYear     55521
LanguageWorkedWith         46539
OrgSize                    77013
UndergradMajor             55105
YearsCodePro               64118
dtype: int64

In [6]:
N = 303590
N

303590

In [7]:
print("Braki")
for i in df.columns:
    print(i,": " , round(100*df[i].isna().sum()/N,2), "%")

Braki
Year :  0.0 %
Hobbyist :  0.01 %
ConvertedComp :  50.21 %
Country :  0.31 %
DatabaseDesireNextYear :  34.07 %
DatabaseWorkedWith :  28.54 %
DevType :  14.71 %
EdLevel :  4.5 %
Employment :  1.92 %
JobSat :  25.61 %
LanguageDesireNextYear :  18.29 %
LanguageWorkedWith :  15.33 %
OrgSize :  25.37 %
UndergradMajor :  18.15 %
YearsCodePro :  21.12 %


In [8]:
df[~df.isna()]

,Year,Hobbyist,ConvertedComp,Country,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,JobSat,LanguageDesireNextYear,LanguageWorkedWith,OrgSize,UndergradMajor,YearsCodePro
0,2020,Yes,NaN,Germany,Microsoft SQL Server,Elasticsearch;Microsoft SQL Server;Oracle,"Developer, desktop or enterprise applications;...","Master's degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",Slightly satisfied,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,2 to 9 employees,"Computer science, computer engineering, or sof...",27
1,2020,No,NaN,United Kingdom,NaN,NaN,"Developer, full-stack;Developer, mobile","Bachelor's degree (B.A., B.S., B.Eng., etc.)",Employed full-time,Very dissatisfied,Python;Swift,JavaScript;Swift,"1,000 to 4,999 employees","Computer science, computer engineering, or sof...",4
2,2020,Yes,NaN,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN
3,2020,Yes,NaN,Albania,NaN,NaN,NaN,"Master's degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,Slightly dissatisfied,NaN,NaN,20 to 99 employees,"Computer science, computer engineering, or sof...",4
4,2020,Yes,NaN,United States,MySQL;PostgreSQL,MySQL;PostgreSQL;Redis;SQLite,NaN,"Bachelor's degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,Java;Ruby;Scala,HTML/CSS;Ruby;SQL,NaN,"Computer science, computer engineering, or sof...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303586,2017,"Yes, I program as a hobby",58000.0,United States,SQL Server,MongoDB; SQL Server; MySQL,Web developer; Developer with a statistics or ...,Bachelor's degree,Employed full-time,3.0,C#; JavaScript; Python; SQL; VB.NET; VBA,C#; Python; R; Ruby; Rust; Scala; TypeScript; ...,100 to 499 employees,A social science,1 to 2 years
303587,2017,No,NaN,Venezuela,MySQL; PostgreSQL,SQL Server; PostgreSQL,NaN,Master's degree,Employed full-time,NaN,Java; SQL,C#; Java; JavaScript; PHP; Python; Ruby; SQL; ...,100 to 499 employees,Computer programming or Web development,NaN
303588,2017,"Yes, I program as a hobby",NaN,Canada,NaN,NaN,Web developer; Systems administrator,Some college/university study without earning ...,Employed full-time,10.0,NaN,NaN,10 to 19 employees,"Information technology, networking, or system ...",Less than a year
303589,2017,"Yes, I program as a hobby",40000.0,United States,MySQL,NaN,Web developer; Mobile developer,Bachelor's degree,Employed full-time,7.0,JavaScript; PHP; Swift,Clojure; Erlang; Haskell,Fewer than 10 employees,Computer science or software engineering,3 to 4 years


In [9]:
N = 303590
N

303590

In [10]:
print("Braki")
for i in df.columns:
    print(i,": " , round(100*df[i].isna().sum()/N,2), "%")

Braki
Year :  0.0 %
Hobbyist :  0.01 %
ConvertedComp :  50.21 %
Country :  0.31 %
DatabaseDesireNextYear :  34.07 %
DatabaseWorkedWith :  28.54 %
DevType :  14.71 %
EdLevel :  4.5 %
Employment :  1.92 %
JobSat :  25.61 %
LanguageDesireNextYear :  18.29 %
LanguageWorkedWith :  15.33 %
OrgSize :  25.37 %
UndergradMajor :  18.15 %
YearsCodePro :  21.12 %


In [11]:
#usuwanie dla ConvertedComp pustych wierszy 
df.dropna(subset = ["ConvertedComp"], inplace=True)
df

,Year,Hobbyist,ConvertedComp,Country,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,JobSat,LanguageDesireNextYear,LanguageWorkedWith,OrgSize,UndergradMajor,YearsCodePro
7,2020,Yes,116000.00000,United States,MongoDB,MariaDB;MySQL;Redis,"Developer, back-end;Developer, desktop or ente...","Bachelor's degree (B.A., B.S., B.Eng., etc.)",Employed full-time,Slightly dissatisfied,JavaScript,Python;SQL,20 to 99 employees,"Computer science, computer engineering, or sof...",13
9,2020,Yes,32315.00000,United Kingdom,Microsoft SQL Server,Microsoft SQL Server,"Database administrator;Developer, full-stack;D...","Master's degree (M.A., M.S., M.Eng., MBA, etc.)",Employed full-time,Very satisfied,HTML/CSS;Java;JavaScript;Python;R;SQL,HTML/CSS;Java;JavaScript;Python;SQL,2 to 9 employees,Mathematics or statistics,4
10,2020,Yes,40070.00000,United Kingdom,Firebase;MongoDB;PostgreSQL,Firebase;MongoDB;PostgreSQL;SQLite,"Developer, back-end;Developer, desktop or ente...","Bachelor's degree (B.A., B.S., B.Eng., etc.)",Employed full-time,Slightly dissatisfied,Go;JavaScript;Swift;TypeScript,C#;JavaScript;Swift,"10,000 or more employees","Computer science, computer engineering, or sof...",2
11,2020,No,14268.00000,Spain,NaN,NaN,"Designer;Developer, front-end",Some college/university study without earning ...,Employed full-time,Very dissatisfied,HTML/CSS;JavaScript,HTML/CSS;JavaScript,2 to 9 employees,Mathematics or statistics,7
12,2020,Yes,38916.00000,Netherlands,NaN,NaN,"Designer;Developer, back-end","Secondary school (e.g. American high school, G...",Employed full-time,Very satisfied,Python,C;JavaScript;Python,2 to 9 employees,NaN,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303570,2017,No,74193.54839,Netherlands,Redis; MySQL,Redis; MySQL; PostgreSQL,Web developer; Machine learning specialist; De...,Bachelor's degree,Employed full-time,7.0,Go; JavaScript; Python,Elixir; JavaScript; Python; Scala,Fewer than 10 employees,Computer science or software engineering,5 to 6 years
303577,2017,"Yes, I contribute to open source projects",107526.88170,Cyprus,MongoDB,MongoDB; MySQL,Mobile developer; DevOps specialist,Secondary school,Employed part-time,10.0,JavaScript; Objective-C; Python; Swift,CoffeeScript; JavaScript; Python; Ruby,Fewer than 10 employees,NaN,5 to 6 years
303581,2017,"Yes, I program as a hobby",32258.06452,France,NaN,NaN,Web developer,Master's degree,Employed full-time,5.0,C#; JavaScript; TypeScript,JavaScript,100 to 499 employees,Computer science or software engineering,1 to 2 years
303586,2017,"Yes, I program as a hobby",58000.00000,United States,SQL Server,MongoDB; SQL Server; MySQL,Web developer; Developer with a statistics or ...,Bachelor's degree,Employed full-time,3.0,C#; JavaScript; Python; SQL; VB.NET; VBA,C#; Python; R; Ruby; Rust; Scala; TypeScript; ...,100 to 499 employees,A social science,1 to 2 years


In [12]:
df["EdLevel"].replace({
"Associate degree (A.A., A.S., etc.)":"Associate degree",
"Bachelor's degree (B.A., B.S., B.Eng., etc.":"Bachelor's degree",
"Bachelor's degree (BA, BS, B.Eng., etc.)":"Bachelor's degree",
"Bachelor‚'s degree (BA, BS, B.Eng., etc.)":"Bachelor's degree",
"Bachelor's degree (B.A., B.S., B.Eng., etc.)":"Bachelor's degree",
"Master's degree (M.A., M.S., M.Eng., MBA, etc.)":"Master's degree",
"Master's degree (MA, MS, M.Eng., MBA, etc.)":"Master's degree",
"Master‚'s degree (MA, MS, M.Eng., MBA, etc.)":"Master's degree",
"Other doctoral degree (Ph.D, Ed.D., etc.)":"Doctoral degree",
"Other doctoral degree (Ph.D., Ed.D., etc.)":"Doctoral degree",
"Professional degree (JD, MD, etc.)":"Professional degree",
"Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)":"Secondary school",
"Some college/university study without earning a bachelor's degree":"Some college/university study without earning a degree"
}, inplace=True)

In [13]:
df

,Year,Hobbyist,ConvertedComp,Country,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,JobSat,LanguageDesireNextYear,LanguageWorkedWith,OrgSize,UndergradMajor,YearsCodePro
7,2020,Yes,116000.00000,United States,MongoDB,MariaDB;MySQL;Redis,"Developer, back-end;Developer, desktop or ente...",Bachelor's degree,Employed full-time,Slightly dissatisfied,JavaScript,Python;SQL,20 to 99 employees,"Computer science, computer engineering, or sof...",13
9,2020,Yes,32315.00000,United Kingdom,Microsoft SQL Server,Microsoft SQL Server,"Database administrator;Developer, full-stack;D...",Master's degree,Employed full-time,Very satisfied,HTML/CSS;Java;JavaScript;Python;R;SQL,HTML/CSS;Java;JavaScript;Python;SQL,2 to 9 employees,Mathematics or statistics,4
10,2020,Yes,40070.00000,United Kingdom,Firebase;MongoDB;PostgreSQL,Firebase;MongoDB;PostgreSQL;SQLite,"Developer, back-end;Developer, desktop or ente...",Bachelor's degree,Employed full-time,Slightly dissatisfied,Go;JavaScript;Swift;TypeScript,C#;JavaScript;Swift,"10,000 or more employees","Computer science, computer engineering, or sof...",2
11,2020,No,14268.00000,Spain,NaN,NaN,"Designer;Developer, front-end",Some college/university study without earning ...,Employed full-time,Very dissatisfied,HTML/CSS;JavaScript,HTML/CSS;JavaScript,2 to 9 employees,Mathematics or statistics,7
12,2020,Yes,38916.00000,Netherlands,NaN,NaN,"Designer;Developer, back-end",Secondary school,Employed full-time,Very satisfied,Python,C;JavaScript;Python,2 to 9 employees,NaN,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303570,2017,No,74193.54839,Netherlands,Redis; MySQL,Redis; MySQL; PostgreSQL,Web developer; Machine learning specialist; De...,Bachelor's degree,Employed full-time,7.0,Go; JavaScript; Python,Elixir; JavaScript; Python; Scala,Fewer than 10 employees,Computer science or software engineering,5 to 6 years
303577,2017,"Yes, I contribute to open source projects",107526.88170,Cyprus,MongoDB,MongoDB; MySQL,Mobile developer; DevOps specialist,Secondary school,Employed part-time,10.0,JavaScript; Objective-C; Python; Swift,CoffeeScript; JavaScript; Python; Ruby,Fewer than 10 employees,NaN,5 to 6 years
303581,2017,"Yes, I program as a hobby",32258.06452,France,NaN,NaN,Web developer,Master's degree,Employed full-time,5.0,C#; JavaScript; TypeScript,JavaScript,100 to 499 employees,Computer science or software engineering,1 to 2 years
303586,2017,"Yes, I program as a hobby",58000.00000,United States,SQL Server,MongoDB; SQL Server; MySQL,Web developer; Developer with a statistics or ...,Bachelor's degree,Employed full-time,3.0,C#; JavaScript; Python; SQL; VB.NET; VBA,C#; Python; R; Ruby; Rust; Scala; TypeScript; ...,100 to 499 employees,A social science,1 to 2 years


In [14]:
df["Hobbyist"].replace({
"Yes, I contribute to open source projects":"Yes",
"Yes, I program as a hobby":"Yes",
"Yes, both":"Yes"}, inplace=True)

In [15]:
df["Country"].replace({
"Azerbaidjan":"Azerbaijan",
"Bosnia-Herzegovina":"Bosnia and Herzegovina",
"CÃ´te d'Ivoire":"Ivory Coast",
"C√¥te d'Ivoire":"Ivory Coast",
"Hong Kong (S.A.R.)":"Hong Kong",
"Iran, Islamic Republic of...":"Iran",
"Lao People's Democratic Republic":"Laos",
"Libyan Arab Jamahiriya":"Libia",
"Republic of Korea":"Korea",
"South Korea":"Korea",
"Saint Vincent & Grenadines":"Saint Vincent and the Grenadines",
"Slovak Republic":"Slovakia",
"Syrian Arab Republic":"Syria",
"Tadjikistan":"Tajikistan",
"United Republic of Tanzania":"Tanzania",
"Venezuela, Bolivarian Republic of...":"Venezuela",
"Viet Nam":"Vietnam",
"Congo, Republic of the...":"Congo",
"Ivory Coast (Cote D'Ivoire)":"Ivory Coast"
}, inplace=True)

In [16]:
df["OrgSize"].replace({"2-9 employees":"Fewer than 10 employees", 
                       "2 to 9 employees":"Fewer than 10 employees", 
                       "Just me - I am a freelancer, sole proprietor, etc.":"1 - freelancer, sole prioprietor, etc."
                      }, inplace=True)

In [17]:
df["Employment"].replace({
    'Retired' : 'not employed',
    'Not employed, and not looking for work' : 'not employed',
    'Not employed, but looking for work': 'not employed',
    }, inplace=True)

df

,Year,Hobbyist,ConvertedComp,Country,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,JobSat,LanguageDesireNextYear,LanguageWorkedWith,OrgSize,UndergradMajor,YearsCodePro
7,2020,Yes,116000.00000,United States,MongoDB,MariaDB;MySQL;Redis,"Developer, back-end;Developer, desktop or ente...",Bachelor's degree,Employed full-time,Slightly dissatisfied,JavaScript,Python;SQL,20 to 99 employees,"Computer science, computer engineering, or sof...",13
9,2020,Yes,32315.00000,United Kingdom,Microsoft SQL Server,Microsoft SQL Server,"Database administrator;Developer, full-stack;D...",Master's degree,Employed full-time,Very satisfied,HTML/CSS;Java;JavaScript;Python;R;SQL,HTML/CSS;Java;JavaScript;Python;SQL,Fewer than 10 employees,Mathematics or statistics,4
10,2020,Yes,40070.00000,United Kingdom,Firebase;MongoDB;PostgreSQL,Firebase;MongoDB;PostgreSQL;SQLite,"Developer, back-end;Developer, desktop or ente...",Bachelor's degree,Employed full-time,Slightly dissatisfied,Go;JavaScript;Swift;TypeScript,C#;JavaScript;Swift,"10,000 or more employees","Computer science, computer engineering, or sof...",2
11,2020,No,14268.00000,Spain,NaN,NaN,"Designer;Developer, front-end",Some college/university study without earning ...,Employed full-time,Very dissatisfied,HTML/CSS;JavaScript,HTML/CSS;JavaScript,Fewer than 10 employees,Mathematics or statistics,7
12,2020,Yes,38916.00000,Netherlands,NaN,NaN,"Designer;Developer, back-end",Secondary school,Employed full-time,Very satisfied,Python,C;JavaScript;Python,Fewer than 10 employees,NaN,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303570,2017,No,74193.54839,Netherlands,Redis; MySQL,Redis; MySQL; PostgreSQL,Web developer; Machine learning specialist; De...,Bachelor's degree,Employed full-time,7.0,Go; JavaScript; Python,Elixir; JavaScript; Python; Scala,Fewer than 10 employees,Computer science or software engineering,5 to 6 years
303577,2017,Yes,107526.88170,Cyprus,MongoDB,MongoDB; MySQL,Mobile developer; DevOps specialist,Secondary school,Employed part-time,10.0,JavaScript; Objective-C; Python; Swift,CoffeeScript; JavaScript; Python; Ruby,Fewer than 10 employees,NaN,5 to 6 years
303581,2017,Yes,32258.06452,France,NaN,NaN,Web developer,Master's degree,Employed full-time,5.0,C#; JavaScript; TypeScript,JavaScript,100 to 499 employees,Computer science or software engineering,1 to 2 years
303586,2017,Yes,58000.00000,United States,SQL Server,MongoDB; SQL Server; MySQL,Web developer; Developer with a statistics or ...,Bachelor's degree,Employed full-time,3.0,C#; JavaScript; Python; SQL; VB.NET; VBA,C#; Python; R; Ruby; Rust; Scala; TypeScript; ...,100 to 499 employees,A social science,1 to 2 years


In [18]:
df.shape

(151172, 15)

In [19]:
df["JobSat"] = df["JobSat"].astype(str)
df["JobSat"].replace({
    'Extremely dissatisfied': '1',
    'Very dissatisfied': '1',
    'Moderately dissatisfied': '1',
    '0.0': '1',
    '1.0': '1',
    '2.0': '1',
    '3.0': '1',
    'Slightly dissatisfied': '2',
    'Neither satisfied nor dissatisfied': '2',
    'Slightly satisfied': '2',
    '4.0': '2',
    '5.0': '2',
    '6.0': '2',
    'Moderately satisfied': '3',
    'Very satisfied': '3',
    'Extremely satisfied': '3',
    '7.0': '3',
    '8.0': '3',
    '9.0': '3',
    '10.0': '3',
    }, inplace=True)

In [20]:
df.drop(["DatabaseDesireNextYear","LanguageDesireNextYear", "UndergradMajor"], axis="columns", inplace=True)

In [21]:
df.shape

(151172, 13)

In [22]:
df.columns

Index(['Year', 'Hobbyist', 'ConvertedComp', 'Country', 'DatabaseWorkedWith',
       'DevType', 'EdLevel', 'Employment', 'JobSat', 'LanguageWorkedWith',
       'OrgSize', 'UndergradMajor', 'YearsCodePro'],
      dtype='object')

In [23]:
d = df["DevType"].str.get_dummies(sep=";")
d

,Data scientist,Database administrator,Desktop applications developer,DevOps specialist,Developer with a statistics or mathematics background,Embedded applications/devices developer,Graphic designer,Graphics programming,Machine learning specialist,Mobile developer,...,Other,Product manager,QA or test developer,Quality assurance engineer,Scientist,Senior executive/VP,Student,System administrator,Systems administrator,Web developer
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303570,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
303577,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303581,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
303586,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [24]:
d["Academic, educator, researcher"] = d["Scientist"] + d["Academic researcher"] +  d["Educator or academic researcher"] + d["Educator"] 
d["Data scientist or ML"] = d["Data scientist"] + d["Data scientist or machine learning specialist"] + d[" Data scientist"] + d["Engineer, data"] + d["Data or business analyst"] + d["Machine learning specialist"]+ d[" Machine learning specialist"] 
d["Database admin"]= d[" Database administrator"] + d["Database administrator"]
d["Desktop app dev"]= d[" Desktop applications developer"] + d["Developer, desktop or enterprise applications"] + d["Desktop applications developer"] + d["Desktop or enterprise applications developer"]
d["System admin"]=d["Systems administrator"] + d["System administrator"]+ d[" Systems administrator"]
d["Developer with stat"] = d[" Developer with a statistics or mathematics background"]+ d["Developer with a statistics or mathematics background"]
d["Back-end dev"] = d["Back-end developer"] + d["Developer, back-end"]
d["Graphic design"]=d[" Graphics programming"] + d["Graphics programming"] + d[" Graphic designer"] + d["Graphic designer"] + d["Designer"]
d["Front-end dev"] = d["Front-end developer"] + d["Developer, front-end"]
d["Full-stack dev"] = d["Full-stack developer"] + d["Developer, full-stack"]
d["Game/graphics dev"] = d["Game or graphics developer"]+d["Developer, game or graphics"]
d["QA/test eng"] = d["QA or test developer"] + d["Developer, QA or test"] + d["Quality assurance engineer"] + d[" Quality assurance engineer"]
d["Mobile dev"] = d[" Mobile developer"] + d["Developer, mobile"] + d["Mobile developer"]
d["DevOps"] =  d[" DevOps specialist"] + d["DevOps specialist"]
d["Embedded Developer"]=d["Embedded applications/devices developer"]+d[" Embedded applications/devices developer"]+d["Embedded applications or devices developer"] +d["Developer, embedded applications or devices"]
d["Other dev"] = d[" Other"] + d["Other"]

In [47]:
d["CEO, CTO, etc"] = d["C-suite executive (CEO, CTO, etc.)"]
d["Engineer of site reliability"] = d["Engineer, site reliability"]
d["Engineering Manager"] = d["Engineering manager"]
d["Marketing or sales Professional"] = d["Marketing or sales professional"]
d["Product Manager"] = d["Product manager"]
d["Senior Executive/VP"] = d["Senior executive/VP"]
d["Students"] = d["Student"]
d["Web developers"]  =  d["Web developer"]

In [48]:
d1 = d.loc[:, 'Academic, educator, researcher':'Web developers']
d1

,"Academic, educator, researcher",Data scientist or ML,Database admin,Desktop app dev,System admin,Developer with stat,Back-end dev,Graphic design,Front-end dev,Full-stack dev,...,Embedded Developer,Other dev,"CEO, CTO, etc",Engineer of site reliability,Engineering Manager,Marketing or sales Professional,Product Manager,Senior Executive/VP,Students,Web developers
7,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,1,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303570,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
303577,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303581,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
303586,0,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


,"Academic, educator, researcher",Data scientist or ML,Database admin,Desktop app dev,System admin,Developer with stat,Back-end dev,Graphic design,Front-end dev,Full-stack dev,...,Embedded Developer,Other dev,"CEO, CTO, etc",Engineer of site reliability,Engineering Manager,Marketing or sales Professional,Product Manager,Senior Executive/VP,Students,Web developers
7,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,1,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303570,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
303577,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303581,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
303586,0,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [50]:
df.drop(columns=["DevType"], axis=1, inplace=True)

In [51]:
df

,Year,Hobbyist,ConvertedComp,Country,DatabaseWorkedWith,EdLevel,Employment,JobSat,LanguageWorkedWith,OrgSize,UndergradMajor,YearsCodePro
7,2020,Yes,116000.00000,United States,MariaDB;MySQL;Redis,Bachelor's degree,Employed full-time,2,Python;SQL,20 to 99 employees,"Computer science, computer engineering, or sof...",13
9,2020,Yes,32315.00000,United Kingdom,Microsoft SQL Server,Master's degree,Employed full-time,3,HTML/CSS;Java;JavaScript;Python;SQL,Fewer than 10 employees,Mathematics or statistics,4
10,2020,Yes,40070.00000,United Kingdom,Firebase;MongoDB;PostgreSQL;SQLite,Bachelor's degree,Employed full-time,2,C#;JavaScript;Swift,"10,000 or more employees","Computer science, computer engineering, or sof...",2
11,2020,No,14268.00000,Spain,NaN,Some college/university study without earning ...,Employed full-time,1,HTML/CSS;JavaScript,Fewer than 10 employees,Mathematics or statistics,7
12,2020,Yes,38916.00000,Netherlands,NaN,Secondary school,Employed full-time,3,C;JavaScript;Python,Fewer than 10 employees,NaN,20
...,...,...,...,...,...,...,...,...,...,...,...,...
303570,2017,No,74193.54839,Netherlands,Redis; MySQL; PostgreSQL,Bachelor's degree,Employed full-time,3,Elixir; JavaScript; Python; Scala,Fewer than 10 employees,Computer science or software engineering,5 to 6 years
303577,2017,Yes,107526.88170,Cyprus,MongoDB; MySQL,Secondary school,Employed part-time,3,CoffeeScript; JavaScript; Python; Ruby,Fewer than 10 employees,NaN,5 to 6 years
303581,2017,Yes,32258.06452,France,NaN,Master's degree,Employed full-time,2,JavaScript,100 to 499 employees,Computer science or software engineering,1 to 2 years
303586,2017,Yes,58000.00000,United States,MongoDB; SQL Server; MySQL,Bachelor's degree,Employed full-time,1,C#; Python; R; Ruby; Rust; Scala; TypeScript; ...,100 to 499 employees,A social science,1 to 2 years


In [55]:
#sklejamy df z d1

df['index1'] = df.index
d1['index1'] = d1.index
pd.merge(df, d1, on="index1",  indicator=True)


,"Academic, educator, researcher",Data scientist or ML,Database admin,Desktop app dev,System admin,Developer with stat,Back-end dev,Graphic design,Front-end dev,Full-stack dev,...,Country,DatabaseWorkedWith,EdLevel,Employment,JobSat,LanguageWorkedWith,OrgSize,UndergradMajor,YearsCodePro,_merge
0,0,0,0,1,0,0,1,0,0,0,...,United States,MariaDB;MySQL;Redis,Bachelor's degree,Employed full-time,2,Python;SQL,20 to 99 employees,"Computer science, computer engineering, or sof...",13,both
1,0,0,1,0,0,0,0,0,0,1,...,United Kingdom,Microsoft SQL Server,Master's degree,Employed full-time,3,HTML/CSS;Java;JavaScript;Python;SQL,Fewer than 10 employees,Mathematics or statistics,4,both
2,0,0,0,1,0,0,1,0,1,0,...,United Kingdom,Firebase;MongoDB;PostgreSQL;SQLite,Bachelor's degree,Employed full-time,2,C#;JavaScript;Swift,"10,000 or more employees","Computer science, computer engineering, or sof...",2,both
3,0,0,0,0,0,0,0,1,1,0,...,Spain,NaN,Some college/university study without earning ...,Employed full-time,1,HTML/CSS;JavaScript,Fewer than 10 employees,Mathematics or statistics,7,both
4,0,0,0,0,0,0,1,1,0,0,...,Netherlands,NaN,Secondary school,Employed full-time,3,C;JavaScript;Python,Fewer than 10 employees,NaN,20,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151167,0,1,0,0,1,1,0,0,0,0,...,Netherlands,Redis; MySQL; PostgreSQL,Bachelor's degree,Employed full-time,3,Elixir; JavaScript; Python; Scala,Fewer than 10 employees,Computer science or software engineering,5 to 6 years,both
151168,0,0,0,0,0,0,0,0,0,0,...,Cyprus,MongoDB; MySQL,Secondary school,Employed part-time,3,CoffeeScript; JavaScript; Python; Ruby,Fewer than 10 employees,NaN,5 to 6 years,both
151169,0,0,0,0,0,0,0,0,0,0,...,France,NaN,Master's degree,Employed full-time,2,JavaScript,100 to 499 employees,Computer science or software engineering,1 to 2 years,both
151170,0,0,1,1,0,1,0,0,0,0,...,United States,MongoDB; SQL Server; MySQL,Bachelor's degree,Employed full-time,1,C#; Python; R; Ruby; Rust; Scala; TypeScript; ...,100 to 499 employees,A social science,1 to 2 years,both


# TO DO

- DatabaseWorkedWith - rozbić, get_dummies
- LanguagesWorkedWith - rozbić get_dummies
- co z Krajem?
- zrobić  model
- Piklowanie

